In [1]:
# Importing in the libraries I need 

import pandas as pd
import sklearn.metrics
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
import warnings 
from xgboost.sklearn import XGBClassifier, XGBRegressor

In [2]:
# Importing the data from the Kaggle page 

sample_submission = pd.DataFrame(pd.read_csv('sample_submission.csv'))
test_feats = pd.DataFrame(pd.read_csv('test_features.csv.zip'))
train_feats = pd.DataFrame(pd.read_csv('train_features.csv.zip'))
train_labels = pd.DataFrame(pd.read_csv('train_labels.csv.zip'))

df = train_feats.append(test_feats)

In [3]:
# Doing some preliminary data observation 

print ('Sample submission array shape:', sample_submission.shape)
print ('\n')
print ('Test features array shape:', test_feats.shape)
print ('\n')
print ('Train features array shape:', train_feats.shape)
print ('\n')
print ('Train labels array shape:', train_labels.shape)
print ('\n')
print ('Percentage of data held out for testing:',(
    len(test_feats) / (len(train_feats)+len(test_feats)))*100,'%')

# So it looks like we have just about 20% of the data held out for testing 


Sample submission array shape: (14358, 2)


Test features array shape: (14358, 40)


Train features array shape: (59400, 40)


Train labels array shape: (59400, 2)


Percentage of data held out for testing: 19.46636297079639 %


In [4]:
# Renaming the dataframes 

X_train = train_feats
X_test = test_feats

y_train = train_labels


In [5]:
pd.set_option('display.max_columns', None)
X_train.head(5)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [6]:
X_train.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],
      dtype='object')

In [7]:
feature_list = X_train.columns

In [8]:
# SCRAP THIS LATER

X_train.subvillage.nunique()

19287

In [9]:
feature_list = feature_list.tolist()

In [10]:
feature_list

['id',
 'amount_tsh',
 'date_recorded',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'wpt_name',
 'num_private',
 'basin',
 'subvillage',
 'region',
 'region_code',
 'district_code',
 'lga',
 'ward',
 'population',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'construction_year',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

In [11]:
X_train.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [12]:
#def feature_precleaner(list_of_feats, df):
 #   feature_name = []
  #  data_type = []
   # null_counts = []
    #null_pct = []
    #unique_vals = []
    #for feature in list_of_feats:
     #   feature_name.append(feature), data_type.append(df[feature].dtypes,
      #  null_counts.append(df[feature].isnull()), null_pct.append(
       #     df[feature].isnull().sum()), unique_vals.append(
        #    df[feature].nunique()), feature_df = pd.DataFrame({'Feature_name':feature_name,
         #                                                      'data_type':data_type,
          #                                                     'null_counts':null_counts,
           #                                                    'null_pct':null_pct,
            #                                                   'unique_vals':unique_vals})return feature_df
        #
    
    

In [13]:
#feature_precleaner(feature_list, X_train)

In [14]:
X_train.describe(include=np.object)

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,59400,55765,55745,59400,59400,59029,59400,59400,59400,56066,59400,55523,31234,56344,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400
unique,356,1897,2145,37400,9,19287,21,125,2092,2,1,12,2696,2,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6
top,2011-03-15,Government Of Tanzania,DWE,none,Lake Victoria,Madukani,Iringa,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,K,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
freq,572,9084,17402,3563,10248,508,5294,2503,307,51011,59400,36793,682,38852,26780,26780,26780,40507,52490,25348,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625


In [15]:
X_train.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [16]:
df['funder'].fillna('unknown', inplace=True)

In [17]:
df['funder'].isna().sum()

0

In [18]:
df['permit'].fillna('unknown', inplace=True)



In [19]:
df['scheme_name'].fillna('unknown', inplace=True)


In [20]:
df['scheme_management'].fillna('unknown', inplace=True)


In [21]:
df['public_meeting'].fillna('unknown', inplace=True)


In [22]:
df['subvillage'].fillna('unknown', inplace=True)


In [23]:
df['installer'].fillna('unknown', inplace=True)

In [24]:
df.isna().sum()

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
scheme_name              0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
s

In [25]:
# A quick glance at the map of Tanzania will reveal A latitudinal range of around 1-ish to 12.5-ish south
# and a longitudinal range of around 25 east to about 40-ish east, so values outside of these must be cleaned

In [26]:
lga_list = df['lga'].unique().tolist()

In [27]:
len(lga_list)

125

In [28]:
lga_list[7]

'Kahama'

In [29]:
# I am going to replace impossible values in longitude and latitude with the mean logitude and latitude of that 
# instance's LGA

In [30]:
df[df['lga'] == 'Kahama']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
7,54551,0.0,2012-10-09,Rwssp,0,DWE,32.620617,-4.226198,Tushirikiane,0,Lake Tanganyika,Nyawishi Center,Shinyanga,17,3,Kahama,Chambo,0,True,GeoData Consultants Ltd,unknown,unknown,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
128,33235,0.0,2012-10-05,Rwssp,0,DWE,32.951559,-3.779393,Nguvu Kazi B,0,Internal,Butondolo,Shinyanga,17,3,Kahama,Jana,0,True,GeoData Consultants Ltd,unknown,unknown,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
133,20673,0.0,2012-10-23,Rwssp,0,DWE,32.544321,-4.076605,Kwa Jacob Shimo,0,Lake Tanganyika,Igalula A,Shinyanga,17,3,Kahama,Ukune,0,True,GeoData Consultants Ltd,unknown,unknown,True,0,other,other,other,wug,user-group,unknown,unknown,milky,milky,insufficient,insufficient,shallow well,shallow well,groundwater,other,other
255,10891,0.0,2012-10-28,Rwssp,0,DWE,32.505150,-3.911655,Kwa Kazumba Justina,0,Lake Tanganyika,Nyandekwa,Shinyanga,17,3,Kahama,Nyandekwa,0,True,GeoData Consultants Ltd,unknown,unknown,True,0,other,other,other,wug,user-group,unknown,unknown,milky,milky,dry,dry,shallow well,shallow well,groundwater,other,other
405,20132,0.0,2013-02-07,Rwssp,0,DWE,32.651853,-3.849545,Kayala,0,Lake Victoria,Makungu,Shinyanga,17,3,Kahama,Mwendakulima,0,True,GeoData Consultants Ltd,unknown,unknown,True,0,nira/tanira,nira/tanira,handpump,private operator,commercial,unknown,unknown,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
630,60433,0.0,2012-10-19,Rwssp,0,DWE,32.261344,-3.853062,Kwa Simoni Maziku,0,Lake Tanganyika,Bukuba A,Shinyanga,17,3,Kahama,Idahina,0,True,GeoData Consultants Ltd,unknown,unknown,True,0,other,other,other,wug,user-group,unknown,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,other,other
690,32699,0.0,2012-10-02,Rwssp,0,RWSSP,32.731288,-3.825468,Songambele,0,Lake Victoria,Lyamalagwa,Shinyanga,17,3,Kahama,Isagehe,0,True,GeoData Consultants Ltd,unknown,unknown,True,0,other,other,other,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,other,other
723,69377,0.0,2013-01-25,World Vision,0,ADP,32.452937,-3.638983,Wichamike,0,Lake Tanganyika,Wichamike,Shinyanga,17,3,Kahama,Ntobo,0,True,GeoData Consultants Ltd,unknown,unknown,True,0,afridev,afridev,handpump,wug,user-group,unknown,unknown,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
736,13866,0.0,2012-10-29,Rwssp,0,DWE,32.648268,-3.733956,Said Mashimba,0,Lake Victoria,Nyashimba,Shinyanga,17,3,Kahama,Kinaga,0,True,GeoData Consultants Ltd,unknown,unknown,True,0,swn 80,swn 80,handpump,wug,user-group,unknown,unknown,milky,milky,dry,dry,machine dbh,borehole,groundwater,hand pump,hand pump
885,19362,0.0,2013-01-23,Rwssp,0,DWE,32.388298,-3.433346,Mwongo,0,Lake Tanganyika,Iponyabukoli,Shinyanga,17,3,Kahama,Lunguya,0,True,GeoData Consultants Ltd,unknown,unknown,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


In [31]:
len_test_df = df[df['lga'] == 'Kahama']

In [32]:
len_test_df['lga']


7        Kahama
128      Kahama
133      Kahama
255      Kahama
405      Kahama
630      Kahama
690      Kahama
723      Kahama
736      Kahama
885      Kahama
902      Kahama
946      Kahama
1003     Kahama
1008     Kahama
1019     Kahama
1028     Kahama
1141     Kahama
1145     Kahama
1181     Kahama
1264     Kahama
1396     Kahama
1469     Kahama
1565     Kahama
1569     Kahama
1619     Kahama
2047     Kahama
2115     Kahama
2170     Kahama
2225     Kahama
2274     Kahama
          ...  
12156    Kahama
12201    Kahama
12277    Kahama
12343    Kahama
12448    Kahama
12552    Kahama
12643    Kahama
12653    Kahama
12873    Kahama
12922    Kahama
12966    Kahama
12990    Kahama
13030    Kahama
13154    Kahama
13472    Kahama
13473    Kahama
13493    Kahama
13568    Kahama
13687    Kahama
13700    Kahama
13719    Kahama
13795    Kahama
13884    Kahama
13976    Kahama
14013    Kahama
14025    Kahama
14045    Kahama
14099    Kahama
14156    Kahama
14343    Kahama
Name: lga, Length: 1036,

In [33]:
no_impossible_coordinates_df = df[(df['longitude'] > 0) & df['latitude'] > 0]

In [34]:
print ('df with impossible coordinates size:', df.shape)
print ('df without impossible coordinates size:', no_impossible_coordinates_df.shape)

df with impossible coordinates size: (73758, 40)
df without impossible coordinates size: (71510, 40)


In [35]:
def lga_center_finder(lga_list, dataframe):
    lga_mean_coordinates = []
    for lga in lga_list:
        lga_df = dataframe[dataframe['lga'] ==lga]
        lga_mean_long = lga_df['longitude'].mean()
        lga_mean_lat = lga_df['latitude'].mean()
        lga_mean_alt = lga_df['gps_height'].mean()
        lga_mean_coordinates.append([lga, lga_mean_long, lga_mean_lat, lga_mean_alt])
    return lga_mean_coordinates 
        
        
    

In [36]:
lga_means_df = pd.DataFrame(lga_center_finder(lga_list, no_impossible_coordinates_df))

In [37]:
lga_means_df.head()

,0,1,2,3
0,Ludewa,34.689386,-9.900529,1512.520468
1,Serengeti,34.521112,-1.810682,1418.806630
2,Simanjiro,36.946443,-3.889944,993.403315
3,Nanyumbu,38.453891,-11.020106,316.605634
4,Karagwe,30.952168,-1.486603,0.000000


In [38]:
lga_means_df.shape

(125, 4)

In [39]:
lga_means_df.columns = ('lga', 'mean_longitude', 'mean_latitude', 'mean_altitude')

In [40]:
lga_means_df.head()

,lga,mean_longitude,mean_latitude,mean_altitude
0,Ludewa,34.689386,-9.900529,1512.520468
1,Serengeti,34.521112,-1.810682,1418.806630
2,Simanjiro,36.946443,-3.889944,993.403315
3,Nanyumbu,38.453891,-11.020106,316.605634
4,Karagwe,30.952168,-1.486603,0.000000


In [41]:
df['lga_mean_longitude'] = df['lga']
df['lga_mean_latitude'] = df['lga']
df['lga_mean_altitude'] = df['lga']
df['lga_mean_longitude'].head(3)

0       Ludewa
1    Serengeti
2    Simanjiro
Name: lga_mean_longitude, dtype: object

In [42]:
from collections import defaultdict


d_1 = defaultdict(list)

for idx, row in lga_means_df.iterrows():
    d_1[row['lga']].append(row['mean_longitude'])

d_1 = [{k: v[0]} for k, v in d_1.items()]


d_2 = defaultdict(list)

for idx, row in lga_means_df.iterrows():
    d_2[row['lga']].append(row['mean_latitude'])

d_2 = [{k: v[0]} for k, v in d_2.items()]


d_3 = defaultdict(list)

for idx, row in lga_means_df.iterrows():
    d_3[row['lga']].append(row['mean_altitude'])

d_3 = [{k: v[0]} for k, v in d_3.items()]

In [43]:
d_1

[{'Ludewa': 34.689385584853824},
 {'Serengeti': 34.52111161303863},
 {'Simanjiro': 36.94644309367402},
 {'Nanyumbu': 38.453891225821614},
 {'Karagwe': 30.952167967843767},
 {'Mkinga': 38.91288792630814},
 {'Shinyanga Rural': 33.145688449785546},
 {'Kahama': 32.499355468030885},
 {'Tabora Urban': 32.79903693523077},
 {'Mkuranga': 39.21037549189113},
 {'Namtumbo': 36.02049160268957},
 {'Maswa': 33.78486939984238},
 {'Siha': 37.06241657110909},
 {'Meatu': 34.268408812363035},
 {'Sumbawanga Rural': 31.72570790352578},
 {'Njombe': 34.745627370051366},
 {'Ukerewe': 33.00740335031743},
 {'Bariadi': 34.17414090013272},
 {'Same': 37.920482309602136},
 {'Kigoma Rural': 29.871234926906418},
 {'Moshi Rural': 37.42682472639851},
 {'Lindi Rural': 39.52102586581739},
 {'Rombo': 37.613198176425726},
 {'Chamwino': 35.97707894852873},
 {'Bagamoyo': 38.613307228329326},
 {'Mafia': 39.735545793941164},
 {'Arusha Rural': 36.7008400879472},
 {'Kyela': 33.87255609737822},
 {'Kondoa': 35.830690945564236},
 {'

In [44]:
d_2 

[{'Ludewa': -9.900528792733919},
 {'Serengeti': -1.8106815510386733},
 {'Simanjiro': -3.88994430320442},
 {'Nanyumbu': -11.02010554230047},
 {'Karagwe': -1.486603270916669},
 {'Mkinga': -4.825748699534881},
 {'Shinyanga Rural': -3.719431995388735},
 {'Kahama': -3.8217674677606177},
 {'Tabora Urban': -4.994938866871797},
 {'Mkuranga': -7.188732301676227},
 {'Namtumbo': -10.631705136706163},
 {'Maswa': -3.182805327231527},
 {'Siha': -3.1893083904000057},
 {'Meatu': -3.3049885767294525},
 {'Sumbawanga Rural': -8.351381060702032},
 {'Njombe': -9.164492914097645},
 {'Ukerewe': -2.0202555648752836},
 {'Bariadi': -2.963342734867257},
 {'Same': -4.316998316428574},
 {'Kigoma Rural': -4.827787207507384},
 {'Moshi Rural': -3.322233678397193},
 {'Lindi Rural': -10.068524273524417},
 {'Rombo': -3.1753403556091},
 {'Chamwino': -6.153449418551717},
 {'Bagamoyo': -6.463476834346799},
 {'Mafia': -7.8848520022352995},
 {'Arusha Rural': -3.3605887894323443},
 {'Kyela': -9.532080073342696},
 {'Kondoa': -

In [45]:
d_3 

[{'Ludewa': 1512.5204678362572},
 {'Serengeti': 1418.8066298342542},
 {'Simanjiro': 993.4033149171271},
 {'Nanyumbu': 316.6056338028169},
 {'Karagwe': 0.0},
 {'Mkinga': 0.0},
 {'Shinyanga Rural': 0.0},
 {'Kahama': 0.0},
 {'Tabora Urban': 0.0},
 {'Mkuranga': 36.438395415472776},
 {'Namtumbo': 871.1101895734597},
 {'Maswa': 0.0},
 {'Siha': 1322.5763636363636},
 {'Meatu': 0.0},
 {'Sumbawanga Rural': 1545.430577223089},
 {'Njombe': 1769.3943481053307},
 {'Ukerewe': 1197.6281179138323},
 {'Bariadi': 1296.7566371681417},
 {'Same': 1028.877938517179},
 {'Kigoma Rural': 1056.1320197044336},
 {'Moshi Rural': 1180.9808429118773},
 {'Lindi Rural': 106.11464968152866},
 {'Rombo': 1406.3547523427042},
 {'Chamwino': 0.0},
 {'Bagamoyo': 114.0134600158353},
 {'Mafia': -23.941176470588236},
 {'Arusha Rural': 1429.482508250825},
 {'Kyela': 0.0},
 {'Kondoa': 0.0},
 {'Kilolo': 1565.4408352668213},
 {'Kibondo': 1333.2615803814713},
 {'Makete': 2053.1788931788933},
 {'Singida Rural': 1516.4829308909243},
 {

In [46]:
dict_1 = {}
for d in d_1:
    dict_1.update(d)
    
dict_2 = {}
for d in d_2:
    dict_2.update(d)
    
dict_3 = {}
for d in d_3:
    dict_3.update(d)
    
    
#unpacked_1 = {}
#for d in d_1:
#    unpacked_1.update(d)
    
#unpacked_2 = {}
#for d in d_2:
#    unpacked_2.update(d)
    
#unpacked_3 = {}
#for d in d_3:
#    unpacked_3.update(d)
    
    
    
d_1 = dict_1
d_2 = dict_2
d_3 = dict_3

In [47]:
d_1

{'Ludewa': 34.689385584853824,
 'Serengeti': 34.52111161303863,
 'Simanjiro': 36.94644309367402,
 'Nanyumbu': 38.453891225821614,
 'Karagwe': 30.952167967843767,
 'Mkinga': 38.91288792630814,
 'Shinyanga Rural': 33.145688449785546,
 'Kahama': 32.499355468030885,
 'Tabora Urban': 32.79903693523077,
 'Mkuranga': 39.21037549189113,
 'Namtumbo': 36.02049160268957,
 'Maswa': 33.78486939984238,
 'Siha': 37.06241657110909,
 'Meatu': 34.268408812363035,
 'Sumbawanga Rural': 31.72570790352578,
 'Njombe': 34.745627370051366,
 'Ukerewe': 33.00740335031743,
 'Bariadi': 34.17414090013272,
 'Same': 37.920482309602136,
 'Kigoma Rural': 29.871234926906418,
 'Moshi Rural': 37.42682472639851,
 'Lindi Rural': 39.52102586581739,
 'Rombo': 37.613198176425726,
 'Chamwino': 35.97707894852873,
 'Bagamoyo': 38.613307228329326,
 'Mafia': 39.735545793941164,
 'Arusha Rural': 36.7008400879472,
 'Kyela': 33.87255609737822,
 'Kondoa': 35.830690945564236,
 'Kilolo': 35.965383789651966,
 'Kibondo': 30.714682952988163

In [48]:
df['lga_mean_longitude'] = df['lga_mean_longitude'].map(d_1)
df['lga_mean_latitude'] = df['lga_mean_latitude'].map(d_2)
df['lga_mean_altitude'] = df['lga_mean_altitude'].map(d_3)

df['lga_mean_longitude'].head(3)

0    34.689386
1    34.521112
2    36.946443
Name: lga_mean_longitude, dtype: float64

In [49]:
d_1

{'Ludewa': 34.689385584853824,
 'Serengeti': 34.52111161303863,
 'Simanjiro': 36.94644309367402,
 'Nanyumbu': 38.453891225821614,
 'Karagwe': 30.952167967843767,
 'Mkinga': 38.91288792630814,
 'Shinyanga Rural': 33.145688449785546,
 'Kahama': 32.499355468030885,
 'Tabora Urban': 32.79903693523077,
 'Mkuranga': 39.21037549189113,
 'Namtumbo': 36.02049160268957,
 'Maswa': 33.78486939984238,
 'Siha': 37.06241657110909,
 'Meatu': 34.268408812363035,
 'Sumbawanga Rural': 31.72570790352578,
 'Njombe': 34.745627370051366,
 'Ukerewe': 33.00740335031743,
 'Bariadi': 34.17414090013272,
 'Same': 37.920482309602136,
 'Kigoma Rural': 29.871234926906418,
 'Moshi Rural': 37.42682472639851,
 'Lindi Rural': 39.52102586581739,
 'Rombo': 37.613198176425726,
 'Chamwino': 35.97707894852873,
 'Bagamoyo': 38.613307228329326,
 'Mafia': 39.735545793941164,
 'Arusha Rural': 36.7008400879472,
 'Kyela': 33.87255609737822,
 'Kondoa': 35.830690945564236,
 'Kilolo': 35.965383789651966,
 'Kibondo': 30.714682952988163

In [50]:
df['lga_mean_longitude']

0        34.689386
1        34.521112
2        36.946443
3        38.453891
4        30.952168
5        38.912888
6        33.145688
7        32.499355
8        32.799037
9        30.952168
10       39.210375
11       36.020492
12       33.784869
13       37.062417
14       34.268409
15       31.725708
16       34.745627
17       34.745627
18       33.007403
19       34.689386
20       34.745627
21       34.174141
22       34.745627
23       37.920482
24       29.871235
25       37.426825
26       39.521026
27       30.952168
28       37.613198
29       37.062417
           ...    
14328    34.521112
14329    38.573921
14330    37.426825
14331    39.304536
14332    37.032270
14333    38.453891
14334    32.023826
14335    32.518288
14336    39.210375
14337    34.880546
14338    37.920482
14339    33.283708
14340    36.882764
14341    34.157363
14342    35.830691
14343    32.499355
14344    39.210375
14345    30.111190
14346    34.157363
14347    38.401759
14348    35.454467
14349    34.

In [51]:
df 

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,lga_mean_longitude,lga_mean_latitude,lga_mean_altitude
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322e+00,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,34.689386,-9.900529,1512.520468
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466e+00,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,unknown,GeoData Consultants Ltd,Other,unknown,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,34.521112,-1.810682,1418.806630
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329e+00,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,36.946443,-3.889944,993.403315
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-1.115530e+01,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,unknown,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,38.453891,-11.020106,316.605634
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359e+00,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,unknown,unknown,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,30.952168,-1.486603,0.000000
5,9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587e+00,Tajiri,0,Pangani,Moa/Mwereme,Tanga,4,8,Mkinga,Moa,1,True,GeoData Consultants Ltd,VWC,Zingibali,True,2009,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe,38.912888,-4.825749,0.000000
6,19816,0.0,2012-10-01,Dwsp,0,DWSP,33.362410,-3.766365e+00,Kwa Ngomho,0,Internal,Ishinabulandi,Shinyanga,17,3,Shinyanga Rural,Samuye,0,True,GeoData Consultants Ltd,VWC,unknown,True,0,swn 80,swn 80,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,33.145688,-3.719432,0.000000
7,54551,0.0,2012-10-09,Rwssp,0,DWE,32.620617,-4.226198e+00,Tushirikiane,0,Lake Tanganyika,Nyawishi Center,Shinyanga,17,3,Kahama,Chambo,0,True,GeoData Consultants Ltd,unknown,unknown,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,32.499355,-3.821767,0.000000
8,53934,0.0,2012-11-03,Wateraid,0,Water Aid,32.711100,-5.146712e+00,Kwa Ramadhan Musa,0,Lake Tanganyika,Imalauduki,Tabora,14,6,Tabora Urban,Itetemia,0,True,GeoData Consultants Ltd,VWC,unknown,True,0,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,salty,salty,seasonal,seasonal,machine dbh,borehole,groundwater,hand pump,hand pump,32.799037,-4.994939,0.000000
9,46144,0.0,2011-08-03,Isingiro Ho,0,A

In [52]:
mean_of_means_long = df['lga_mean_longitude'].mean()

In [53]:
df['lga_mean_latitude'].mean()

-5.8159486066241595

In [54]:
df['lga_mean_latitude'].isna().sum()

603

In [55]:
df['lga_mean_longitude'].isna().sum()

603

In [56]:
df.loc[df['longitude'] == 0, "longitude"] = df['lga_mean_longitude'].loc[df['longitude'] == 0]
df['longitude'].min()

29.6071219

In [57]:
df['longitude'].isna().sum()

603

In [58]:
df.loc[df['latitude'] == 0, "latitude"] = df['lga_mean_latitude'].loc[df['latitude'] == 0]
df['latitude'].min()

-11.64944018

In [59]:
df['latitude'].isna().sum()

0

In [60]:
df['longitude'] = df['longitude'].fillna(mean_of_means_long)

In [61]:
# Now the impossible values of 0 in longitude and latitude have been replaced by the average longitude and 
# latitude of that instance's lga 

In [62]:
# The features named, "quantity" and "quantity_group" are exactly the same, so I will delete quantity_group


df =  df.drop('quantity_group',1)

In [63]:
# Thanks to a little bit of domain knowledge, I now know that source_class, source_type, and source are all 
# descriptors of the source with source falling under source_type which falls under source_class, so analyzing 
# source_type or source_class will provide no more granularity or distinction than using only source 

df = df.drop('source_class',1)
df = df.drop('source_type',1)




In [64]:
# Like the various source descriptors, waterpoint_type is also a more granular and specific descriptor which
# eliminates the need to include waterpoint_type_group

df = df.drop('waterpoint_type_group',1)

In [65]:
# quality_group is another redundant and less-specific quality descriptor relative to water_quality


df = df.drop('quality_group',1)

In [66]:
# At first glance, the values of payment and payment_type look different with 'never pay' and 'unknown' being 
# the only congruent values, but upon further inspection, you will find that the other values each only map to 
# exactly one value in the other column, so you could really delete either column while preserving the data's 
# insight 

df = df.drop('payment_type',1)

In [67]:
df.shape

(73758, 37)

In [68]:
df.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'water_quality',
       'quantity', 'source', 'waterpoint_type', 'lga_mean_longitude',
       'lga_mean_latitude', 'lga_mean_altitude'],
      dtype='object')

In [69]:
# management_group is another descriptor column that only generalizes the more specific data in the management 
# column 

df = df.drop('management_group',1)

In [70]:
df.shape

(73758, 36)

In [71]:
# In the case of extraction data, extraction_type_group and extraction_type_class are also both redundant

df = df.drop('extraction_type_group',1)
df = df.drop('extraction_type_class',1)

In [72]:
df.shape 

(73758, 34)

In [73]:
# recorded_by only has one value in the whole column 

df = df.drop('recorded_by',1)

In [74]:
df.shape 

(73758, 33)

In [75]:
# replacing the zero values in construction year with the year value that most closely reflects the label 
# proportions of the zero years

df.construction_year = df.construction_year.replace({0:1986})

In [76]:
# Finding and implementing a solid solution for permit looks like it would only yield marginal results according 
# to my pivot table, so I am just going to drop it alltogether. 

df = df.drop('permit',1)

In [77]:
df.shape 

(73758, 32)

In [78]:
df.head(10)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,lga_mean_longitude,lga_mean_latitude,lga_mean_altitude
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,VWC,Roman,1999,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,34.689386,-9.900529,1512.520468
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,unknown,Other,unknown,2010,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,34.521112,-1.810682,1418.806630
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,VWC,Nyumba ya mungu pipe scheme,2009,gravity,vwc,pay per bucket,soft,enough,dam,communal standpipe multiple,36.946443,-3.889944,993.403315
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,VWC,unknown,1986,submersible,vwc,never pay,soft,dry,machine dbh,communal standpipe multiple,38.453891,-11.020106,316.605634
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,unknown,unknown,1986,gravity,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,30.952168,-1.486603,0.000000
5,9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Tajiri,0,Pangani,Moa/Mwereme,Tanga,4,8,Mkinga,Moa,1,True,VWC,Zingibali,2009,submersible,vwc,pay per bucket,salty,enough,other,communal standpipe multiple,38.912888,-4.825749,0.000000
6,19816,0.0,2012-10-01,Dwsp,0,DWSP,33.362410,-3.766365,Kwa Ngomho,0,Internal,Ishinabulandi,Shinyanga,17,3,Shinyanga Rural,Samuye,0,True,VWC,unknown,1986,swn 80,vwc,never pay,soft,enough,machine dbh,hand pump,33.145688,-3.719432,0.000000
7,54551,0.0,2012-10-09,Rwssp,0,DWE,32.620617,-4.226198,Tushirikiane,0,Lake Tanganyika,Nyawishi Center,Shinyanga,17,3,Kahama,Chambo,0,True,unknown,unknown,1986,nira/tanira,wug,unknown,milky,enough,shallow well,hand pump,32.499355,-3.821767,0.000000
8,53934,0.0,2012-11-03,Wateraid,0,Water Aid,32.711100,-5.146712,Kwa Ramadhan Musa,0,Lake Tanganyika,Imalauduki,Tabora,14,6,Tabora Urban,Itetemia,0,True,VWC,unknown,1986,india mark ii,vwc,never pay,salty,seasonal,machine dbh,hand pump,32.799037,-4.994939,0.000000
9,46144,0.0,2011-08-03,Isingiro Ho,0,Artisan,30.626991,-1.257051,Kwapeto,0,Lake Victoria,Mkonomre,Kagera,18,1,Karagwe,Kaisho,0,True,unknown,unknown,1986,nira/tanira,vwc,never pay,soft,enough,shallow well,hand pump,30.952168,-1.486603,0.000000


In [79]:
df['month'] = pd.to_datetime(df.date_recorded).dt.month
df['year'] = pd.to_datetime(df.date_recorded).dt.year
df['day_of_week'] = pd.to_datetime(df.date_recorded).dt.weekday 

In [80]:
df['day'] = pd.to_datetime(df.date_recorded).dt.day 

In [81]:
from scipy.spatial import ConvexHull

def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371000):
    """
    Vectorized great circle distance between two points
    (lat, lon) specified in decimal degrees or in radians)
    Returns units in meters
    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))


def append_dist_to_group_center(df, group_col='basin'):
    lats = {}
    lons = {}

    grouped = df.groupby(group_col)
    for group_name, group_df in grouped:
        points = list(zip(group_df.latitude, group_df.longitude))

        hull = ConvexHull(points)
        form = [points[i] for i in hull.vertices]

        lats[group_name] = np.mean(hull.points[hull.vertices,0])
        lons[group_name] = np.mean(hull.points[hull.vertices,1])

    centroids_df = (pd.DataFrame([lats, lons]).T
                      .rename(columns={0:f'{group_col}_lat', 1:f'{group_col}_lon'}))

    df = df.merge(centroids_df, left_on=group_col, right_index=True)

    df[f'dist_to_{group_col}'] = haversine(df.latitude, df.longitude,
                                           df[f'{group_col}_lat'], df[f'{group_col}_lon'])
    return df

In [82]:
 df = append_dist_to_group_center(df, group_col='basin')

In [83]:
df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,lga_mean_longitude,lga_mean_latitude,lga_mean_altitude,month,year,day_of_week,day,basin_lat,basin_lon,dist_to_basin
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,VWC,Roman,1999,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,34.689386,-9.900529,1512.520468,3,2011,0,14,-10.025022,34.583075,43171.953815
11,50409,200.0,2013-02-18,Danida,1062,DANIDA,35.770258,-10.574175,Kwa Alid Nchimbi,0,Lake Nyasa,Ngondombwito,Ruvuma,10,5,Namtumbo,Msindo,250,True,WUG,unknown,1987,swn 80,wug,pay when scheme fails,soft,insufficient,shallow well,hand pump,36.020492,-10.631705,871.110190,2,2013,0,18,-10.025022,34.583075,143519.186699
19,18274,500.0,2011-02-22,Danida,1763,Danid,34.508967,-9.894412,none,0,Lake Nyasa,Manyanya,Iringa,11,5,Ludewa,Mawengi,1,True,VWC,Makanj,1992,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,34.689386,-9.900529,1512.520468,2,2011,1,22,-10.025022,34.583075,16637.123016
20,48375,200.0,2011-02-27,Twe,2216,TWE,34.473430,-9.594990,none,0,Lake Nyasa,Majengo,Iringa,11,4,Njombe,Matola,4,True,VWC,Kidabu,2008,gravity,vwc,pay monthly,soft,enough,spring,communal standpipe,34.745627,-9.164493,1769.394348,2,2011,6,27,-10.025022,34.583075,49303.508670
33,12402,0.0,2011-03-11,Undp,1638,DWE,34.478356,-9.715910,none,0,Lake Nyasa,Lupanga A,Iringa,11,5,Ludewa,Lupanga,70,True,VWC,UNDP,1992,gravity,vwc,pay when scheme fails,soft,seasonal,spring,communal standpipe,34.689386,-9.900529,1512.520468,3,2011,4,11,-10.025022,34.583075,36235.633805
36,67359,0.0,2011-07-16,Danida,0,Central government,33.854345,-9.638520,Office Ya Kijiji,0,Lake Nyasa,Ndanganyika,Mbeya,12,3,Kyela,Katumbasongwe,0,True,VWC,Ngana water supplied scheme,1986,gravity,vwc,never pay,soft,dry,spring,communal standpipe,33.872556,-9.532080,0.000000,7,2011,5,16,-10.025022,34.583075,90672.955806
41,12796,500.0,2011-03-12,Government Of Tanzania,2469,Commu,33.927902,-9.204649,none,0,Lake Nyasa,Kilunduwe,Iringa,11,3,Makete,Ipelele,0,True,VWC,Misiwa,1986,gravity,vwc,pay when scheme fails,soft,enough,spring,communal standpipe,34.157363,-9.307761,2053.178893,3,2011,5,12,-10.025022,34.583075,116105.855102
47,13620,0.0,2011-07-27,unknown,0,unknown,33.540607,-9.172905,Mahakamani,0,Lake Nyasa,Mpandapanda,Mbeya,12,4,Rungwe,Kiwira,0,True,VWC,K,1986,gravity,vwc,never pay,soft,enough,spring,communal standpipe,33.713526,-9.247229,0.000000,7,2011,2,27,-10.025022,34.583075,148461.007176
52,10745,0.0,2011-07-31,Kkkt_makwale,0,KKKT _ Konde and DWE,33.958163,-9.448122,Kwa Abeli Mwamakula,0,Lake Nyasa,Isimba,Mbeya,12,3,Kyela,Makwale,0,True,VWC,Makwale water supplied sche,1986,gravity,vwc,never pay,soft,enough,spring,communal standpipe,33.872556,-9.532080,0.000000,7,2011,6,31,-10.025022,34.583075,93836.590365
80,19415,0.0,2011-03-02,Roman,2328,Commu,34.284476,-9.271565,none,0,Lake Nyasa,Ikovo,Iringa,11,3,Makete,Mang'oto,0,True,VWC,Malemb,2002,gravity,vwc,never pay,soft,enough,spring,communal standpipe,34.157363,-9.307761,2053.178893,3,2011,2,2,-10.025022,34.583075,89947.856584


In [84]:
df = append_dist_to_group_center(df, group_col='district_code')

In [85]:
df 

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,lga_mean_longitude,lga_mean_latitude,lga_mean_altitude,month,year,day_of_week,day,basin_lat,basin_lon,dist_to_basin,district_code_lat,district_code_lon,dist_to_district_code
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,VWC,Roman,1999,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,34.689386,-9.900529,1512.520468,3,2011,0,14,-10.025022,34.583075,43171.953815,-5.685452,35.883063,475316.224030
11,50409,200.0,2013-02-18,Danida,1062,DANIDA,35.770258,-10.574175,Kwa Alid Nchimbi,0,Lake Nyasa,Ngondombwito,Ruvuma,10,5,Namtumbo,Msindo,250,True,WUG,unknown,1987,swn 80,wug,pay when scheme fails,soft,insufficient,shallow well,hand pump,36.020492,-10.631705,871.110190,2,2013,0,18,-10.025022,34.583075,143519.186699,-5.685452,35.883063,543742.876834
19,18274,500.0,2011-02-22,Danida,1763,Danid,34.508967,-9.894412,none,0,Lake Nyasa,Manyanya,Iringa,11,5,Ludewa,Mawengi,1,True,VWC,Makanj,1992,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,34.689386,-9.900529,1512.520468,2,2011,1,22,-10.025022,34.583075,16637.123016,-5.685452,35.883063,491877.864542
33,12402,0.0,2011-03-11,Undp,1638,DWE,34.478356,-9.715910,none,0,Lake Nyasa,Lupanga A,Iringa,11,5,Ludewa,Lupanga,70,True,VWC,UNDP,1992,gravity,vwc,pay when scheme fails,soft,seasonal,spring,communal standpipe,34.689386,-9.900529,1512.520468,3,2011,4,11,-10.025022,34.583075,36235.633805,-5.685452,35.883063,474132.818170
135,24012,0.0,2011-03-17,District Council,2050,DWE,34.688597,-9.696226,none,0,Lake Nyasa,Ligelango,Iringa,11,5,Ludewa,Madope,50,True,VWC,Distri,2007,gravity,vwc,pay when scheme fails,soft,enough,spring,communal standpipe,34.689386,-9.900529,1512.520468,3,2011,3,17,-10.025022,34.583075,38344.591144,-5.685452,35.883063,464987.611474
213,43098,0.0,2011-04-04,unknown,0,unknown,33.242441,-9.319142,Kwa Daison,0,Lake Nyasa,Isanga,Mbeya,12,5,Ileje,Ndola,0,True,VWC,unknown,1986,gravity,vwc,unknown,unknown,dry,river,communal standpipe,33.195325,-9.395021,0.000000,4,2011,0,4,-10.025022,34.583075,166599.784793,-5.685452,35.883063,497965.557739
332,17111,0.0,2011-03-27,unknown,0,unknown,32.947525,-9.324812,Kwandimbwa,0,Lake Nyasa,Nambigili,Mbeya,12,5,Ileje,Mbebe,0,True,VWC,unknown,1986,nira/tanira,vwc,unknown,soft,seasonal,shallow well,hand pump,33.195325,-9.395021,0.000000,3,2011,6,27,-10.025022,34.583075,195454.116525,-5.685452,35.883063,518128.716738
389,7007,500.0,2011-02-22,Danida,1694,Danid,34.503924,-9.873567,none,0,Lake Nyasa,Kiwanja,Iringa,11,5,Ludewa,Mawengi,95,True,VWC,Jongoj,1992,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,34.689386,-9.900529,1512.520468,2,2011,1,22,-10.025022,34.583075,18941.167363,-5.685452,35.883063,489846.182214
415,3627,0.0,2011-03-11,Undp,1680,DWE,34.479721,-9.721475,none,0,Lake Nyasa,Lupanga A,Iringa,11,5,Ludewa,Lupanga,35,True,VWC,UNDP,1992,gravity,vwc,pay when scheme fails,soft,dry,spring,communal standpipe,34.689386,-9.900529,1512.520468,3,2011,4,11,-10.025022,34.583075,35601.336743,-5.685452,35.883063,474668.416997
467,66382,0.0,2013-02-18,Abasia,1066,DWE,35.764098,-10.479270,Shuleni,0,Lake Nyasa,Ukombozi,Ruvuma,10,5,Namtumbo,Msindo,375,True,VWC,Majimingi,1994,gravity,vwc,never pay,soft,insufficient,river,communal standpipe,36.020492,-10.631705,871.110190,2,2013,0,18,-10.025022,34.583075,138747.200658,-5.685452,35.883063,533209.018125


In [86]:
df = append_dist_to_group_center(df, group_col='region')

In [87]:
df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,lga_mean_longitude,lga_mean_latitude,lga_mean_altitude,month,year,day_of_week,day,basin_lat,basin_lon,dist_to_basin,district_code_lat,district_code_lon,dist_to_district_code,region_lat,region_lon,dist_to_region
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,VWC,Roman,1999,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,34.689386,-9.900529,1512.520468,3,2011,0,14,-10.025022,34.583075,43171.953815,-5.685452,35.883063,475316.224030,-8.839757,35.246760,118000.983201
19,18274,500.0,2011-02-22,Danida,1763,Danid,34.508967,-9.894412,none,0,Lake Nyasa,Manyanya,Iringa,11,5,Ludewa,Mawengi,1,True,VWC,Makanj,1992,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,34.689386,-9.900529,1512.520468,2,2011,1,22,-10.025022,34.583075,16637.123016,-5.685452,35.883063,491877.864542,-8.839757,35.246760,142494.479661
33,12402,0.0,2011-03-11,Undp,1638,DWE,34.478356,-9.715910,none,0,Lake Nyasa,Lupanga A,Iringa,11,5,Ludewa,Lupanga,70,True,VWC,UNDP,1992,gravity,vwc,pay when scheme fails,soft,seasonal,spring,communal standpipe,34.689386,-9.900529,1512.520468,3,2011,4,11,-10.025022,34.583075,36235.633805,-5.685452,35.883063,474132.818170,-8.839757,35.246760,128848.436371
135,24012,0.0,2011-03-17,District Council,2050,DWE,34.688597,-9.696226,none,0,Lake Nyasa,Ligelango,Iringa,11,5,Ludewa,Madope,50,True,VWC,Distri,2007,gravity,vwc,pay when scheme fails,soft,enough,spring,communal standpipe,34.689386,-9.900529,1512.520468,3,2011,3,17,-10.025022,34.583075,38344.591144,-5.685452,35.883063,464987.611474,-8.839757,35.246760,113233.248971
389,7007,500.0,2011-02-22,Danida,1694,Danid,34.503924,-9.873567,none,0,Lake Nyasa,Kiwanja,Iringa,11,5,Ludewa,Mawengi,95,True,VWC,Jongoj,1992,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,34.689386,-9.900529,1512.520468,2,2011,1,22,-10.025022,34.583075,18941.167363,-5.685452,35.883063,489846.182214,-8.839757,35.246760,140913.729132
415,3627,0.0,2011-03-11,Undp,1680,DWE,34.479721,-9.721475,none,0,Lake Nyasa,Lupanga A,Iringa,11,5,Ludewa,Lupanga,35,True,VWC,UNDP,1992,gravity,vwc,pay when scheme fails,soft,dry,spring,communal standpipe,34.689386,-9.900529,1512.520468,3,2011,4,11,-10.025022,34.583075,35601.336743,-5.685452,35.883063,474668.416997,-8.839757,35.246760,129218.895932
577,42350,3000.0,2011-03-17,Danida,1912,DANID,34.793393,-9.729017,none,0,Lake Nyasa,Mjimwema,Iringa,11,5,Ludewa,Madilu,90,True,VWC,DANIDA,1993,gravity,vwc,pay annually,soft,dry,spring,communal standpipe,34.689386,-9.900529,1512.520468,3,2011,3,17,-10.025022,34.583075,40176.858394,-5.685452,35.883063,465373.426330,-8.839757,35.246760,110691.764146
621,4636,500.0,2011-02-22,Danida,1674,Danid,34.493414,-9.845683,none,0,Lake Nyasa,Sambala,Iringa,11,5,Ludewa,Mawengi,1,True,VWC,Jongoj,1992,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,34.689386,-9.900529,1512.520468,2,2011,1,22,-10.025022,34.583075,22228.509726,-5.685452,35.883063,487263.598246,-8.839757,35.246760,139080.185054
673,63908,0.0,2011-03-11,Undp,1782,MLADE,34.480107,-9.687040,none,0,Lake Nyasa,Ikulungilo,Iringa,11,5,Ludewa,Lupanga,200,True,VWC,UNDP,2004,gravity,vwc,pay when scheme fails,soft,enough,river,communal standpipe,34.689386,-9.900529,1512.520468,3,2011,4,11,-10.025022,34.583075,39238.407523,-5.685452,35.883063,471038.128242,-8.839757,35.246760,126312.861167
927,28993,3000.0,2011-03-07,Roman,1445,DWE,34.666082,-10.117379,none,0,Lake Nyasa,Kilimahewa,Iringa,11,5,Ludewa,Ludewa,75,True,Water Board,Roman,2001,gravity,water board,pay monthly,soft,enough,river,communal standpipe,34.689386,-9.900529,1512.520468,3,2011,0,7,-10.025022,34.583075,13713.175916,-5.6

In [88]:
df.isna().sum()

id                         0
amount_tsh                 0
date_recorded              0
funder                     0
gps_height                 0
installer                  0
longitude                  0
latitude                   0
wpt_name                   0
num_private                0
basin                      0
subvillage                 0
region                     0
region_code                0
district_code              0
lga                        0
ward                       0
population                 0
public_meeting             0
scheme_management          0
scheme_name                0
construction_year          0
extraction_type            0
management                 0
payment                    0
water_quality              0
quantity                   0
source                     0
waterpoint_type            0
lga_mean_longitude       603
lga_mean_latitude        603
lga_mean_altitude        603
month                      0
year                       0
day_of_week   

In [89]:
df = df.drop('lga_mean_longitude',1)


In [90]:
df = df.drop('lga_mean_latitude',1)


In [91]:
df = df.drop('lga_mean_altitude',1)

In [92]:
df.isna().sum()

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
scheme_management        0
scheme_name              0
construction_year        0
extraction_type          0
management               0
payment                  0
water_quality            0
quantity                 0
source                   0
waterpoint_type          0
month                    0
year                     0
day_of_week              0
day                      0
basin_lat                0
basin_lon                0
dist_to_basin            0
district_code_lat        0
d

In [93]:
!pip install pygeohash 
import pygeohash as gh

In [94]:
df['geohash'] = df.apply(lambda x: gh.encode(x.latitude, x.longitude, precision=8), axis=1)

In [95]:
df 

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,month,year,day_of_week,day,basin_lat,basin_lon,dist_to_basin,district_code_lat,district_code_lon,dist_to_district_code,region_lat,region_lon,dist_to_region,geohash
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,VWC,Roman,1999,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,3,2011,0,14,-10.025022,34.583075,43171.953815,-5.685452,35.883063,475316.224030,-8.839757,35.246760,118000.983201,ky0zgj9n
19,18274,500.0,2011-02-22,Danida,1763,Danid,34.508967,-9.894412,none,0,Lake Nyasa,Manyanya,Iringa,11,5,Ludewa,Mawengi,1,True,VWC,Makanj,1992,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,2,2011,1,22,-10.025022,34.583075,16637.123016,-5.685452,35.883063,491877.864542,-8.839757,35.246760,142494.479661,ky0x9qb8
33,12402,0.0,2011-03-11,Undp,1638,DWE,34.478356,-9.715910,none,0,Lake Nyasa,Lupanga A,Iringa,11,5,Ludewa,Lupanga,70,True,VWC,UNDP,1992,gravity,vwc,pay when scheme fails,soft,seasonal,spring,communal standpipe,3,2011,4,11,-10.025022,34.583075,36235.633805,-5.685452,35.883063,474132.818170,-8.839757,35.246760,128848.436371,ky288x62
135,24012,0.0,2011-03-17,District Council,2050,DWE,34.688597,-9.696226,none,0,Lake Nyasa,Ligelango,Iringa,11,5,Ludewa,Madope,50,True,VWC,Distri,2007,gravity,vwc,pay when scheme fails,soft,enough,spring,communal standpipe,3,2011,3,17,-10.025022,34.583075,38344.591144,-5.685452,35.883063,464987.611474,-8.839757,35.246760,113233.248971,ky28v6g7
389,7007,500.0,2011-02-22,Danida,1694,Danid,34.503924,-9.873567,none,0,Lake Nyasa,Kiwanja,Iringa,11,5,Ludewa,Mawengi,95,True,VWC,Jongoj,1992,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,2,2011,1,22,-10.025022,34.583075,18941.167363,-5.685452,35.883063,489846.182214,-8.839757,35.246760,140913.729132,ky0xc4sf
415,3627,0.0,2011-03-11,Undp,1680,DWE,34.479721,-9.721475,none,0,Lake Nyasa,Lupanga A,Iringa,11,5,Ludewa,Lupanga,35,True,VWC,UNDP,1992,gravity,vwc,pay when scheme fails,soft,dry,spring,communal standpipe,3,2011,4,11,-10.025022,34.583075,35601.336743,-5.685452,35.883063,474668.416997,-8.839757,35.246760,129218.895932,ky288w72
577,42350,3000.0,2011-03-17,Danida,1912,DANID,34.793393,-9.729017,none,0,Lake Nyasa,Mjimwema,Iringa,11,5,Ludewa,Madilu,90,True,VWC,DANIDA,1993,gravity,vwc,pay annually,soft,dry,spring,communal standpipe,3,2011,3,17,-10.025022,34.583075,40176.858394,-5.685452,35.883063,465373.426330,-8.839757,35.246760,110691.764146,ky28xszu
621,4636,500.0,2011-02-22,Danida,1674,Danid,34.493414,-9.845683,none,0,Lake Nyasa,Sambala,Iringa,11,5,Ludewa,Mawengi,1,True,VWC,Jongoj,1992,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,2,2011,1,22,-10.025022,34.583075,22228.509726,-5.685452,35.883063,487263.598246,-8.839757,35.246760,139080.185054,ky0xbztk
673,63908,0.0,2011-03-11,Undp,1782,MLADE,34.480107,-9.687040,none,0,Lake Nyasa,Ikulungilo,Iringa,11,5,Ludewa,Lupanga,200,True,VWC,UNDP,2004,gravity,vwc,pay when scheme fails,soft,enough,river,communal standpipe,3,2011,4,11,-10.025022,34.583075,39238.407523,-5.685452,35.883063,471038.128242,-8.839757,35.246760,126312.861167,ky28bse8
927,28993,3000.0,2011-03-07,Roman,1445,DWE,34.666082,-10.117379,none,0,Lake Nyasa,Kilimahewa,Iringa,11,5,Ludewa,Ludewa,75,True,Water Board,Roman,2001,gravity,water board,pay monthly,soft,enough,river,communal standpipe,3,2011,0,7,-10.025022,34.583075,13713.175916,-5.685452,35.883063,510701.497001,-8.839757,35.246760,155686.610323,ky0wky5j


In [96]:
df.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
scheme_management         object
scheme_name               object
construction_year          int64
extraction_type           object
management                object
payment                   object
water_quality             object
quantity                  object
source                    object
waterpoint_type           object
month                      int64
year      

In [97]:
df.nunique()

id                       73758
amount_tsh                 102
date_recorded              369
funder                    2139
gps_height                2455
installer                 2407
longitude                71420
latitude                 71417
wpt_name                 45600
num_private                 68
basin                        9
subvillage               21389
region                      21
region_code                 27
district_code               20
lga                        125
ward                      2098
population                1127
public_meeting               3
scheme_management           13
scheme_name               2869
construction_year           54
extraction_type             18
management                  12
payment                      7
water_quality                8
quantity                     5
source                      10
waterpoint_type              7
month                       12
year                         6
day_of_week                  7
day     

In [98]:
df 

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,month,year,day_of_week,day,basin_lat,basin_lon,dist_to_basin,district_code_lat,district_code_lon,dist_to_district_code,region_lat,region_lon,dist_to_region,geohash
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,VWC,Roman,1999,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,3,2011,0,14,-10.025022,34.583075,43171.953815,-5.685452,35.883063,475316.224030,-8.839757,35.246760,118000.983201,ky0zgj9n
19,18274,500.0,2011-02-22,Danida,1763,Danid,34.508967,-9.894412,none,0,Lake Nyasa,Manyanya,Iringa,11,5,Ludewa,Mawengi,1,True,VWC,Makanj,1992,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,2,2011,1,22,-10.025022,34.583075,16637.123016,-5.685452,35.883063,491877.864542,-8.839757,35.246760,142494.479661,ky0x9qb8
33,12402,0.0,2011-03-11,Undp,1638,DWE,34.478356,-9.715910,none,0,Lake Nyasa,Lupanga A,Iringa,11,5,Ludewa,Lupanga,70,True,VWC,UNDP,1992,gravity,vwc,pay when scheme fails,soft,seasonal,spring,communal standpipe,3,2011,4,11,-10.025022,34.583075,36235.633805,-5.685452,35.883063,474132.818170,-8.839757,35.246760,128848.436371,ky288x62
135,24012,0.0,2011-03-17,District Council,2050,DWE,34.688597,-9.696226,none,0,Lake Nyasa,Ligelango,Iringa,11,5,Ludewa,Madope,50,True,VWC,Distri,2007,gravity,vwc,pay when scheme fails,soft,enough,spring,communal standpipe,3,2011,3,17,-10.025022,34.583075,38344.591144,-5.685452,35.883063,464987.611474,-8.839757,35.246760,113233.248971,ky28v6g7
389,7007,500.0,2011-02-22,Danida,1694,Danid,34.503924,-9.873567,none,0,Lake Nyasa,Kiwanja,Iringa,11,5,Ludewa,Mawengi,95,True,VWC,Jongoj,1992,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,2,2011,1,22,-10.025022,34.583075,18941.167363,-5.685452,35.883063,489846.182214,-8.839757,35.246760,140913.729132,ky0xc4sf
415,3627,0.0,2011-03-11,Undp,1680,DWE,34.479721,-9.721475,none,0,Lake Nyasa,Lupanga A,Iringa,11,5,Ludewa,Lupanga,35,True,VWC,UNDP,1992,gravity,vwc,pay when scheme fails,soft,dry,spring,communal standpipe,3,2011,4,11,-10.025022,34.583075,35601.336743,-5.685452,35.883063,474668.416997,-8.839757,35.246760,129218.895932,ky288w72
577,42350,3000.0,2011-03-17,Danida,1912,DANID,34.793393,-9.729017,none,0,Lake Nyasa,Mjimwema,Iringa,11,5,Ludewa,Madilu,90,True,VWC,DANIDA,1993,gravity,vwc,pay annually,soft,dry,spring,communal standpipe,3,2011,3,17,-10.025022,34.583075,40176.858394,-5.685452,35.883063,465373.426330,-8.839757,35.246760,110691.764146,ky28xszu
621,4636,500.0,2011-02-22,Danida,1674,Danid,34.493414,-9.845683,none,0,Lake Nyasa,Sambala,Iringa,11,5,Ludewa,Mawengi,1,True,VWC,Jongoj,1992,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,2,2011,1,22,-10.025022,34.583075,22228.509726,-5.685452,35.883063,487263.598246,-8.839757,35.246760,139080.185054,ky0xbztk
673,63908,0.0,2011-03-11,Undp,1782,MLADE,34.480107,-9.687040,none,0,Lake Nyasa,Ikulungilo,Iringa,11,5,Ludewa,Lupanga,200,True,VWC,UNDP,2004,gravity,vwc,pay when scheme fails,soft,enough,river,communal standpipe,3,2011,4,11,-10.025022,34.583075,39238.407523,-5.685452,35.883063,471038.128242,-8.839757,35.246760,126312.861167,ky28bse8
927,28993,3000.0,2011-03-07,Roman,1445,DWE,34.666082,-10.117379,none,0,Lake Nyasa,Kilimahewa,Iringa,11,5,Ludewa,Ludewa,75,True,Water Board,Roman,2001,gravity,water board,pay monthly,soft,enough,river,communal standpipe,3,2011,0,7,-10.025022,34.583075,13713.175916,-5.685452,35.883063,510701.497001,-8.839757,35.246760,155686.610323,ky0wky5j


In [99]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()

#enc.fit(df['COL1'])
#df['COL1'] = enc.transform(df['col1'])



#df = df.drop('id',1)
df = df.drop('date_recorded',1)


enc.fit(df['funder'])
df['funder'] = enc.transform(df['funder'])



enc.fit(df['installer'])
df['installer'] = enc.transform(df['installer'])

df = df.drop('wpt_name',1)

enc.fit(df['basin'])
df['basin'] = enc.transform(df['basin'])



df = df.drop('subvillage',1)
df = df.drop('region',1)


enc.fit(df['lga'])
df['lga'] = enc.transform(df['lga'])


df = df.drop('ward',1)
df = df.drop('public_meeting',1)

enc.fit(df['scheme_management'])
df['scheme_management'] = enc.transform(df['scheme_management'])


df = df.drop('scheme_name',1)


enc.fit(df['extraction_type'])
df['extraction_type'] = enc.transform(df['extraction_type'])


enc.fit(df['management'])
df['management'] = enc.transform(df['management'])


enc.fit(df['payment'])
df['payment'] = enc.transform(df['payment'])


enc.fit(df['water_quality'])
df['water_quality'] = enc.transform(df['water_quality'])


enc.fit(df['quantity'])
df['quantity'] = enc.transform(df['quantity'])


enc.fit(df['source'])
df['source'] = enc.transform(df['source'])


enc.fit(df['waterpoint_type'])
df['waterpoint_type'] = enc.transform(df['waterpoint_type'])


df = df.drop('geohash',1)

train = pd.merge(train_labels, df, on='id', how='inner')
test = pd.merge(sample_submission, df, on='id', how='inner')

final_feat_list = df.columns
final_feat_list = final_feat_list.tolist()
final_feat_list

['id',
 'amount_tsh',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'num_private',
 'basin',
 'region_code',
 'district_code',
 'lga',
 'population',
 'scheme_management',
 'construction_year',
 'extraction_type',
 'management',
 'payment',
 'water_quality',
 'quantity',
 'source',
 'waterpoint_type',
 'month',
 'year',
 'day_of_week',
 'day',
 'basin_lat',
 'basin_lon',
 'dist_to_basin',
 'district_code_lat',
 'district_code_lon',
 'dist_to_district_code',
 'region_lat',
 'region_lon',
 'dist_to_region']

In [104]:
param_grid = {
   'xgbclassifier__n_estimators': [100],
   'xgbclassifier__gamma': [0.5, 1, 2],
   'xgbclassifier__objective': ['multi:softprob'],
   'xgbclassifier__booster': ['gbtree'],
   'xgbclassifier__nrounds': ['min.error.idx'],
   'xgbclassifier__num_class': [4],
   'xgbclassifier__maximize': [False],
   'xgbclassifier__eval_metric': ['merror'],
   'xgbclassifier__eta': [.05, .1, .2, .25, .3],
   'xgbclassifier__max_depth': [14, 16, 18],
   'xgbclassifier__colsample_bytree': [.25, .3, .4],
   'xgbclassifier__reg_alpha': [0],
   'xgbclassifier__reg_lambda': [1, 5],
   'xgbclassifier__min_child_weight': [1, 3],
    'xgbclassifier__silent': [False],
    'xgbclassifier__n_jobs':[-1],
    
}

pipe = make_pipeline(XGBClassifier())




In [107]:
y_train = train.status_group
X_train = train.drop('status_group',1)


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696e+00,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214e+00,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344e+00,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672e+00,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-1.095041e+01,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
5,52449,0.0,2013-03-04,Government Of Tanzania,1685,DWE,36.685279,-3.302420e+00,Masaga,0,Pangani,Masaga,Arusha,2,2,Arusha Rural,Ilkiding'a,200,True,GeoData Consultants Ltd,VWC,Timbolo sambasha TPRI pipe line,True,1990,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
6,24806,0.0,2011-03-02,Government Of Tanzania,550,Gover,36.398041,-7.541382e+00,none,0,Rufiji,Kichangani,Iringa,11,7,Kilolo,Mahenge,600,True,GeoData Consultants Ltd,VWC,NaN,True,2007,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,salty,salty,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
7,28965,0.0,2013-01-25,Finw,234,FinW,39.607420,-1.089379e+01,Kwa Mkwaa,0,Ruvuma / Southern Coast,Ilala,Mtwara,9,4,Tandahimba,Maundo,1,True,GeoData Consultants Ltd,Water Board,Borehole,True,1982,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
8,36301,30.0,2013-01-23,Unicef,584,LGA,39.262951,-1.082359e+01,Kwa Mzee Mpini,0,Ruvuma / Southern Coast,Bondeni,Mtwara,90,33,Newala,Mnekachi,40,True,GeoData Consultants Ltd,VWC,MWS,False,1997,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
9,54122,0.0,2013-03-18,Lawatefuka Water Supply,1083,Lawatefuka water sup,37.096108,-3.251754e+00,Kwa Flora Daud,0,Pangani,Donyomuruak,Kilimanjaro,3,7,Siha,Siha Kati,1,True,GeoData Consultants Ltd,Water Board,BL Erasto Laizer,True,2003,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [110]:



grid_search_accuracy = GridSearchCV(pipe, param_grid, scoring='accuracy', cv=3, verbose=10,
                                    n_jobs=-1, return_train_score=True)



In [111]:
grid_search_accuracy.fit(X_train, y_train)





Fitting 3 folds for each of 540 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   51.8s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   54.9s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed: 14.0min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done 197 tasks      | elapsed: 1

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('xgbclassifier', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'xgbclassifier__n_estimators': [100], 'xgbclassifier__gamma': [0.5, 1, 2], 'xgbclassifier__objective': ['multi:softprob'], 'xgbclassifier__booster': ['gbtree'], 'xgbclassifier__nrounds': ['min.error.idx'], 'xgbclassifier__num_class': [4], 'xgbclassifier__maximize': [False], 'xgbclassifie...ssifier__min_child_weight': [1, 3], 'xgbclassifier__silent': [False], 'xgbclassifier__n_jobs': [-1]},
       pre_dispatch='2*n_job

In [112]:
X_test = test.drop('status_group',1)


In [113]:
preds = grid_search_accuracy.predict(X_test)



In [114]:
grid_results = pd.DataFrame(grid_search_accuracy.cv_results_)

In [115]:
grid_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_xgbclassifier__booster,param_xgbclassifier__colsample_bytree,param_xgbclassifier__eta,param_xgbclassifier__eval_metric,param_xgbclassifier__gamma,param_xgbclassifier__max_depth,param_xgbclassifier__maximize,param_xgbclassifier__min_child_weight,param_xgbclassifier__n_estimators,param_xgbclassifier__n_jobs,param_xgbclassifier__nrounds,param_xgbclassifier__num_class,param_xgbclassifier__objective,param_xgbclassifier__reg_alpha,param_xgbclassifier__reg_lambda,param_xgbclassifier__silent,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,40.193496,0.257478,4.634474,0.064080,gbtree,0.25,0.05,merror,0.5,14,False,1,100,-1,min.error.idx,4,multi:softprob,0,1,False,"{'xgbclassifier__booster': 'gbtree', 'xgbclass...",0.812374,0.813030,0.808131,0.811178,0.002171,111,0.962601,0.965631,0.964722,0.964318,0.001270
1,41.022885,0.090129,4.508777,0.122060,gbtree,0.25,0.05,merror,0.5,14,False,1,100,-1,min.error.idx,4,multi:softprob,0,5,False,"{'xgbclassifier__booster': 'gbtree', 'xgbclass...",0.809697,0.811818,0.807475,0.809663,0.001773,321,0.928182,0.929773,0.930606,0.929520,0.001006
2,39.522105,0.364797,4.497737,0.290933,gbtree,0.25,0.05,merror,0.5,14,False,3,100,-1,min.error.idx,4,multi:softprob,0,1,False,"{'xgbclassifier__booster': 'gbtree', 'xgbclass...",0.812879,0.812374,0.809091,0.811448,0.001679,51,0.945884,0.946288,0.947374,0.946515,0.000629
3,39.497648,0.090831,3.897599,0.286674,gbtree,0.25,0.05,merror,0.5,14,False,3,100,-1,min.error.idx,4,multi:softprob,0,5,False,"{'xgbclassifier__booster': 'gbtree', 'xgbclass...",0.810859,0.812071,0.806010,0.809646,0.002618,331,0.919874,0.920934,0.923131,0.921313,0.001357
4,49.185723,0.219739,4.727820,0.349960,gbtree,0.25,0.05,merror,0.5,16,False,1,100,-1,min.error.idx,4,multi:softprob,0,1,False,"{'xgbclassifier__booster': 'gbtree', 'xgbclass...",0.811263,0.812323,0.808384,0.810657,0.001664,216,0.980404,0.981187,0.980859,0.980816,0.000321
5,49.442384,0.223310,5.080349,0.141509,gbtree,0.25,0.05,merror,0.5,16,False,1,100,-1,min.error.idx,4,multi:softprob,0,5,False,"{'xgbclassifier__booster': 'gbtree', 'xgbclass...",0.812525,0.811919,0.807727,0.810724,0.002133,196,0.943510,0.944343,0.944040,0.943965,0.000344
6,47.898516,0.182681,5.204668,0.107154,gbtree,0.25,0.05,merror,0.5,16,False,3,100,-1,min.error.idx,4,multi:softprob,0,1,False,"{'xgbclassifier__booster': 'gbtree', 'xgbclass...",0.811667,0.814444,0.806919,0.811010,0.003107,136,0.962374,0.963131,0.963359,0.962955,0.000421
7,47.619966,0.259975,4.751124,0.236806,gbtree,0.25,0.05,merror,0.5,16,False,3,100,-1,min.error.idx,4,multi:softprob,0,5,False,"{'xgbclassifier__booster': 'gbtree', 'xgbclass...",0.811869,0.810556,0.806768,0.809731,0.002163,306,0.934545,0.935455,0.935859,0.935286,0.000549
8,56.326267,0.323220,6.083854,0.406711,gbtree,0.25,0.05,merror,0.5,18,False,1,100,-1,min.error.idx,4,multi:softprob,0,1,False,"{'xgbclassifier__booster': 'gbtree', 'xgbclass...",0.810455,0.812677,0.805303,0.809478,0.003088,346,0.988535,0.989495,0.988434,0.988822,0.000478
9,55.569698,0.359052,5.571900,0.162129,gbtree,0.25,0.05,merror,0.5,18,False,1,100,-1,min.error.idx,4,multi:softprob,0,5,False,"{'xgbclassifier__booster': 'gbtree', 'xgbclass...",0.811616,0.813333,0.807929,0.810960,0.002255,146,0.952551,0.952677,0.952702,0.952643,0.000066


In [116]:
preds 

array(['non functional', 'functional', 'functional', ..., 'functional',
       'functional', 'non functional'], dtype=object)

In [117]:
sample_submission['status_group'] = preds 

In [118]:
sample_submission

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
5,52449,functional
6,24806,non functional
7,28965,non functional
8,36301,non functional
9,54122,functional


In [119]:
sample_submission.to_csv('well_i_guess_we_will_see.csv', index=False)